In [1]:
D.<x> = PolynomialRing(QQ)
R = PolynomialRing(QQ,'x')
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H4(n,m): # this is the definition in the published interpolations paper (2021):
    # def. 8.4
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.

def H4strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H4(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer


def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
     #if argument is an S type then 'polynomial' takes no argument.

def H6strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return ((ddstep1(n,m)-ddstep2(n,m))/1728)

def deltaDaggerStrike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def deltaDagger2(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))

# def deltaDaggerStrike(n,m):
 #   return S(((H4strike(n,3)^3-H6strike(n,3)^2)/1728)).truncate(n+1)
    
def deltaDagger2Strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger2(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr



print("ok")

ok


In [2]:
data=[]
import time
start=time.time()/60
for m in [3..1203]:
    poly=H6(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/24sept22no2.txt','wb') # H6 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 200, 0.2616325840353966]
[4, 200, 0.4389880858361721]
[5, 200, 0.7912262864410877]
[6, 200, 0.9542607516050339]
[7, 200, 1.3468129374086857]
[8, 200, 1.6441184841096401]
[9, 200, 2.069517269730568]
[10, 200, 2.336151883006096]
[11, 200, 2.763600632548332]
[12, 200, 3.130292646586895]
[13, 200, 3.585161168128252]
[14, 200, 3.8797726705670357]
[15, 200, 4.390367701649666]
[16, 200, 4.747877597808838]
[17, 200, 5.234898250550032]
[18, 200, 5.5596346370875835]
[19, 200, 6.032913003116846]
[20, 200, 6.457231052219868]
[21, 200, 6.987989619374275]
[22, 200, 7.30693593621254]
[23, 200, 7.790721852332354]
[24, 200, 8.234562534838915]
[25, 200, 8.75962920114398]
[26, 200, 9.122010603547096]
[27, 200, 9.654974721372128]
[28, 200, 10.109375033527613]
[29, 200, 10.614894770085812]
[30, 200, 11.012355368584394]
[31, 200, 11.515254702419043]
[32, 200, 11.930758584290743]
[33, 200, 12.512196268886328]
[34, 200, 12.887050103396177]
[35, 200, 13.482817884534597]
[36, 200, 13.975628636777401]
[37, 2

[277, 200, 163.82346544787288]
[278, 200, 164.37040576711297]
[279, 200, 165.1275083012879]
[280, 200, 165.8341271840036]
[281, 200, 166.5392679311335]
[282, 200, 167.13750933483243]
[283, 200, 167.85821451991796]
[284, 200, 168.50953805074096]
[285, 200, 169.317191619426]
[286, 200, 169.88718051835895]
[287, 200, 170.65901433303952]
[288, 200, 171.30808668211102]
[289, 200, 172.06768398359418]
[290, 200, 172.6535854525864]
[291, 200, 173.41209598630667]
[292, 200, 174.04498375207186]
[293, 200, 174.77041465044022]
[294, 200, 175.38363958522677]
[295, 200, 176.14220624789596]
[296, 200, 176.78749085590243]
[297, 200, 177.55135640129447]
[298, 200, 178.09125580266118]
[299, 200, 178.84815751761198]
[300, 200, 179.55879660323262]
[301, 200, 180.32319905236363]
[302, 200, 180.85827971994877]
[303, 200, 181.62773833423853]
[304, 200, 182.27326676622033]
[305, 200, 183.04567246884108]
[306, 200, 183.6450878828764]
[307, 200, 184.3825293891132]
[308, 200, 185.0873415991664]
[309, 200, 185.86

[546, 200, 357.4779072031379]
[547, 200, 358.2372756190598]
[548, 200, 358.9569663852453]
[549, 200, 359.7999281845987]
[550, 200, 360.47745560109615]
[551, 200, 361.2854486890137]
[552, 200, 362.0383473858237]
[553, 200, 362.8472862020135]
[554, 200, 363.45527383312583]
[555, 200, 364.3350444547832]
[556, 200, 365.02887113392353]
[557, 200, 365.8354577869177]
[558, 200, 366.4888855665922]
[559, 200, 367.312930367887]
[560, 200, 368.06799718365073]
[561, 200, 368.9343880712986]
[562, 200, 369.5418296866119]
[563, 200, 370.32847253605723]
[564, 200, 371.093788549304]
[565, 200, 371.91481621935964]
[566, 200, 372.5496348030865]
[567, 200, 373.4155530035496]
[568, 200, 374.1266596317291]
[569, 200, 374.8860325664282]
[570, 200, 375.5811961144209]
[571, 200, 376.36296673864126]
[572, 200, 377.0997177027166]
[573, 200, 377.93195071816444]
[574, 200, 378.5746644884348]
[575, 200, 379.4264334663749]
[576, 200, 380.14909860119224]
[577, 200, 380.94265346974134]
[578, 200, 381.5606083162129]
[5

[818, 200, 567.4869251996279]
[819, 200, 568.3967986851931]
[820, 200, 569.2067561857402]
[821, 200, 570.0280904173851]
[822, 200, 570.71771203354]
[823, 200, 571.5464624017477]
[824, 200, 572.3040179349482]
[825, 200, 573.2257128842175]
[826, 200, 573.9039069525898]
[827, 200, 574.7311723344028]
[828, 200, 575.5196490176022]
[829, 200, 576.3481564819813]
[830, 200, 577.0284863188863]
[831, 200, 577.8747783303261]
[832, 200, 578.6120545677841]
[833, 200, 579.4798700362444]
[834, 200, 580.176321618259]
[835, 200, 581.0152930170298]
[836, 200, 581.8069337010384]
[837, 200, 582.6694524362683]
[838, 200, 583.3291927054524]
[839, 200, 584.1398733146489]
[840, 200, 584.991791933775]
[841, 200, 585.8091836050153]
[842, 200, 586.4366092197597]
[843, 200, 587.2960241995752]
[844, 200, 588.0280937179923]
[845, 200, 588.9172160848975]
[846, 200, 589.6062309034169]
[847, 200, 590.4795774035156]
[848, 200, 591.1983653195202]
[849, 200, 592.0503493845463]
[850, 200, 592.7455456033349]
[851, 200, 593

[1090, 200, 787.088083203882]
[1091, 200, 787.9443434365094]
[1092, 200, 788.8052518181503]
[1093, 200, 789.6584376506507]
[1094, 200, 790.3239544183016]
[1095, 200, 791.2470983490348]
[1096, 200, 792.0107200145721]
[1097, 200, 792.8684620037675]
[1098, 200, 793.5831109024584]
[1099, 200, 794.4780299179256]
[1100, 200, 795.318705484271]
[1101, 200, 796.2117562703788]
[1102, 200, 796.8864020854235]
[1103, 200, 797.7317508868873]
[1104, 200, 798.5436288006604]
[1105, 200, 799.4669469706714]
[1106, 200, 800.1819508001208]
[1107, 200, 801.0991211496294]
[1108, 200, 801.8876435831189]
[1109, 200, 802.7524429485202]
[1110, 200, 803.5145603530109]
[1111, 200, 804.4009410515428]
[1112, 200, 805.1740420162678]
[1113, 200, 806.1087652519345]
[1114, 200, 806.7938203364611]
[1115, 200, 807.7134974226356]
[1116, 200, 808.5524427518249]
[1117, 200, 809.4314245507121]
[1118, 200, 810.1308513358235]
[1119, 200, 811.0170000009239]
[1120, 200, 811.8382683843374]
[1121, 200, 812.6987687051296]
[1122, 200

In [3]:
data=[]
import time
start=time.time()/60
for m in [1204..2403]:
    poly=H6(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/25sept22no1.txt','wb') # H6 series (date typo)
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[1204, 200, 0.8238958865404129]
[1205, 200, 1.7626532688736916]
[1206, 200, 2.4998037852346897]
[1207, 200, 3.4233929701149464]
[1208, 200, 4.226199202239513]
[1209, 200, 5.212466701865196]
[1210, 200, 5.9707087725400925]
[1211, 200, 6.890312422066927]
[1212, 200, 7.753071002662182]
[1213, 200, 8.647605769336224]
[1214, 200, 9.370969101786613]
[1215, 200, 10.349708817899227]
[1216, 200, 11.158944837749004]
[1217, 200, 12.03750041872263]
[1218, 200, 12.823291786015034]
[1219, 200, 13.710373017936945]
[1220, 200, 14.550003219395876]
[1221, 200, 15.498194739222527]
[1222, 200, 16.20114891976118]
[1223, 200, 17.089831449091434]
[1224, 200, 17.959448017179966]
[1225, 200, 18.935408234596252]
[1226, 200, 19.629303868860006]
[1227, 200, 20.54473726823926]
[1228, 200, 21.356110885739326]
[1229, 200, 22.24446725472808]
[1230, 200, 23.025517899543047]
[1231, 200, 23.88956780359149]
[1232, 200, 24.754638217389584]
[1233, 200, 25.688640486449003]
[1234, 200, 26.410126119852066]
[1235, 200, 27.3714

[1465, 200, 223.05666628479958]
[1466, 200, 223.776779435575]
[1467, 200, 224.69682573899627]
[1468, 200, 225.5174032896757]
[1469, 200, 226.41813671961427]
[1470, 200, 227.23290568590164]
[1471, 200, 228.12384590134025]
[1472, 200, 228.8935829512775]
[1473, 200, 229.81237838417292]
[1474, 200, 230.53557700291276]
[1475, 200, 231.4809820689261]
[1476, 200, 232.33214225247502]
[1477, 200, 233.24971428513527]
[1478, 200, 233.93907411769032]
[1479, 200, 234.88679615035653]
[1480, 200, 235.73177545517683]
[1481, 200, 236.61369586735964]
[1482, 200, 237.39645078405738]
[1483, 200, 238.29097193479538]
[1484, 200, 239.14594546705484]
[1485, 200, 240.1397937051952]
[1486, 200, 240.85798991844058]
[1487, 200, 241.75431840121746]
[1488, 200, 242.60117586702108]
[1489, 200, 243.47128318622708]
[1490, 200, 244.19966848567128]
[1491, 200, 245.17294643819332]
[1492, 200, 245.98782160133123]
[1493, 200, 246.887736517936]
[1494, 200, 247.63192426785827]
[1495, 200, 248.6060621701181]
[1496, 200, 249.4

[1727, 200, 451.0166580863297]
[1728, 200, 451.8566225208342]
[1729, 200, 452.8429923988879]
[1730, 200, 453.6273032538593]
[1731, 200, 454.56827046722174]
[1732, 200, 455.40683713555336]
[1733, 200, 456.31512661650777]
[1734, 200, 457.09720270335674]
[1735, 200, 458.05592571944]
[1736, 200, 458.9505390152335]
[1737, 200, 459.9252637028694]
[1738, 200, 460.7101848721504]
[1739, 200, 461.6587543524802]
[1740, 200, 462.58726393803954]
[1741, 200, 463.5387393012643]
[1742, 200, 464.2872450873256]
[1743, 200, 465.271267067641]
[1744, 200, 466.06441985443234]
[1745, 200, 467.0141465701163]
[1746, 200, 467.77615885436535]
[1747, 200, 468.6938908845186]
[1748, 200, 469.56641306728125]
[1749, 200, 470.58742145076394]
[1750, 200, 471.36762980371714]
[1751, 200, 472.3140211850405]
[1752, 200, 473.17409943789244]
[1753, 200, 474.1068850196898]
[1754, 200, 474.8653931543231]
[1755, 200, 475.9048804529011]
[1756, 200, 476.74964371696115]
[1757, 200, 477.7140596359968]
[1758, 200, 478.50334123522043

[1992, 200, 684.4876006022096]
[1993, 200, 685.4087442196906]
[1994, 200, 686.1243189685047]
[1995, 200, 687.1709523499012]
[1996, 200, 687.997394438833]
[1997, 200, 688.9165750518441]
[1998, 200, 689.6925027854741]
[1999, 200, 690.6227855570614]
[2000, 200, 691.4820105880499]
[2001, 200, 692.4639415033162]
[2002, 200, 693.2524011880159]
[2003, 200, 694.1503114998341]
[2004, 200, 695.0251692831516]
[2005, 200, 695.9710080698133]
[2006, 200, 696.7173563204706]
[2007, 200, 697.6772914864123]
[2008, 200, 698.5292324014008]
[2009, 200, 699.4914714694023]
[2010, 200, 700.3168444521725]
[2011, 200, 701.228388119489]
[2012, 200, 702.0720452852547]
[2013, 200, 703.0592860691249]
[2014, 200, 703.8001481667161]
[2015, 200, 704.7829986512661]
[2016, 200, 705.6705939695239]
[2017, 200, 706.5981576554477]
[2018, 200, 707.3408871851861]
[2019, 200, 708.2971600666642]
[2020, 200, 709.16039269045]
[2021, 200, 710.069377951324]
[2022, 200, 710.8329429700971]
[2023, 200, 711.7842385694385]
[2024, 200, 7

[2258, 200, 920.3944491185248]
[2259, 200, 921.3805249035358]
[2260, 200, 922.2756400853395]
[2261, 200, 923.2718799859285]
[2262, 200, 924.0759444534779]
[2263, 200, 925.024491854012]
[2264, 200, 925.874813400209]
[2265, 200, 926.9192561395466]
[2266, 200, 927.6969756186008]
[2267, 200, 928.6453556008637]
[2268, 200, 929.5811430215836]
[2269, 200, 930.5210401527584]
[2270, 200, 931.304339338094]
[2271, 200, 932.2809365876019]
[2272, 200, 933.0921618379653]
[2273, 200, 933.9935618191957]
[2274, 200, 934.7923780046403]
[2275, 200, 935.8138334862888]
[2276, 200, 936.6900123022497]
[2277, 200, 937.694229837507]
[2278, 200, 938.45669330284]
[2279, 200, 939.3899756222963]
[2280, 200, 940.3329447172582]
[2281, 200, 941.2666767537594]
[2282, 200, 942.0530539192259]
[2283, 200, 943.0351365059614]
[2284, 200, 943.8847982175648]
[2285, 200, 944.8582117855549]
[2286, 200, 945.658617220819]
[2287, 200, 946.6013707853854]
[2288, 200, 947.4826856516302]
[2289, 200, 948.4885874353349]
[2290, 200, 949

In [2]:
data=[]
import time
start=time.time()/60
for m in [2404..3610]:
    poly=H6(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/28sept22no1.txt','wb') # H6 series (date typo)
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[2404, 200, 0.848883718252182]
[2405, 200, 1.8108874186873436]
[2406, 200, 2.5860639847815037]
[2407, 200, 3.5086042508482933]
[2408, 200, 4.371610552072525]
[2409, 200, 5.380045600235462]
[2410, 200, 6.150921750813723]
[2411, 200, 7.098175685852766]
[2412, 200, 7.997469283640385]
[2413, 200, 8.963465068489313]
[2414, 200, 9.703870117664337]
[2415, 200, 10.750581003725529]
[2416, 200, 11.573403351008892]
[2417, 200, 12.492281217128038]
[2418, 200, 13.28802827000618]
[2419, 200, 14.200567219406366]
[2420, 200, 15.095801819115877]
[2421, 200, 16.048783138394356]
[2422, 200, 16.809257853776217]
[2423, 200, 17.720043569803238]
[2424, 200, 18.5912104845047]
[2425, 200, 19.540025383234024]
[2426, 200, 20.272792033851147]
[2427, 200, 21.23348870128393]
[2428, 200, 22.073094252496958]
[2429, 200, 23.025848288089037]
[2430, 200, 23.838553585112095]
[2431, 200, 24.804591435939074]
[2432, 200, 25.610088516026735]
[2433, 200, 26.58592863753438]
[2434, 200, 27.321261439472437]
[2435, 200, 28.282317

[2665, 200, 232.86542250588536]
[2666, 200, 233.62132338434458]
[2667, 200, 234.61136490106583]
[2668, 200, 235.4715956710279]
[2669, 200, 236.39298030361533]
[2670, 200, 237.22545525059104]
[2671, 200, 238.13647535070777]
[2672, 200, 238.97309803590178]
[2673, 200, 239.95100730285048]
[2674, 200, 240.7363687902689]
[2675, 200, 241.69571790471673]
[2676, 200, 242.59134021773934]
[2677, 200, 243.51273168623447]
[2678, 200, 244.27198860049248]
[2679, 200, 245.26512371748686]
[2680, 200, 246.15474170073867]
[2681, 200, 247.1157477349043]
[2682, 200, 247.91343729943037]
[2683, 200, 248.86846830323339]
[2684, 200, 249.74062930047512]
[2685, 200, 250.76072549819946]
[2686, 200, 251.53210085257888]
[2687, 200, 252.4609269350767]
[2688, 200, 253.34269701689482]
[2689, 200, 254.26336968690157]
[2690, 200, 255.04729107022285]
[2691, 200, 256.05325108394027]
[2692, 200, 256.9024545326829]
[2693, 200, 257.8365380652249]
[2694, 200, 258.6171695999801]
[2695, 200, 259.6343393176794]
[2696, 200, 260.

[2927, 200, 467.8679325841367]
[2928, 200, 468.7546174712479]
[2929, 200, 469.69623240455985]
[2930, 200, 470.4935071989894]
[2931, 200, 471.4708274193108]
[2932, 200, 472.3296694383025]
[2933, 200, 473.3202802725136]
[2934, 200, 474.1046591848135]
[2935, 200, 475.0937297157943]
[2936, 200, 475.940067820251]
[2937, 200, 476.93408335000277]
[2938, 200, 477.69857381656766]
[2939, 200, 478.6364279203117]
[2940, 200, 479.5999411009252]
[2941, 200, 480.5467881523073]
[2942, 200, 481.31379203498363]
[2943, 200, 482.2922624349594]
[2944, 200, 483.11088478192687]
[2945, 200, 484.10223688930273]
[2946, 200, 484.87619910016656]
[2947, 200, 485.8317206837237]
[2948, 200, 486.69075708836317]
[2949, 200, 487.6823957338929]
[2950, 200, 488.47037410363555]
[2951, 200, 489.44164365157485]
[2952, 200, 490.33818378672004]
[2953, 200, 491.28379798308015]
[2954, 200, 492.0451036170125]
[2955, 200, 493.0666092187166]
[2956, 200, 493.92346769943833]
[2957, 200, 494.87211948260665]
[2958, 200, 495.7041330710

[3192, 200, 707.3428539521992]
[3193, 200, 708.2901482358575]
[3194, 200, 709.0520762354136]
[3195, 200, 710.0744159407914]
[3196, 200, 710.9629748351872]
[3197, 200, 711.8947952203453]
[3198, 200, 712.7038538344204]
[3199, 200, 713.6771833859384]
[3200, 200, 714.5371493697166]
[3201, 200, 715.5515927150846]
[3202, 200, 716.3009711503983]
[3203, 200, 717.2540725022554]
[3204, 200, 718.1241199336946]
[3205, 200, 719.1104892343283]
[3206, 200, 719.8908380679786]
[3207, 200, 720.8726885169744]
[3208, 200, 721.7191321328282]
[3209, 200, 722.6493227854371]
[3210, 200, 723.4895105324686]
[3211, 200, 724.4645349867642]
[3212, 200, 725.372638553381]
[3213, 200, 726.3990209884942]
[3214, 200, 727.1738606877625]
[3215, 200, 728.1343283876777]
[3216, 200, 729.0384702160954]
[3217, 200, 729.9780287370086]
[3218, 200, 730.7207732200623]
[3219, 200, 731.7188435830176]
[3220, 200, 732.6636270545423]
[3221, 200, 733.6049881987274]
[3222, 200, 734.3995636180043]
[3223, 200, 735.3636759854853]
[3224, 20

[3458, 200, 950.3673819713295]
[3459, 200, 951.3620646484196]
[3460, 200, 952.252432603389]
[3461, 200, 953.197536803782]
[3462, 200, 954.0000101178885]
[3463, 200, 954.9523891843855]
[3464, 200, 955.8327063694596]
[3465, 200, 956.903619684279]
[3466, 200, 957.6806841865182]
[3467, 200, 958.6375910006464]
[3468, 200, 959.5653648860753]
[3469, 200, 960.5086303055286]
[3470, 200, 961.3114733174443]
[3471, 200, 962.3036099672318]
[3472, 200, 963.161477919668]
[3473, 200, 964.119283836335]
[3474, 200, 964.925704088062]
[3475, 200, 965.9180993698537]
[3476, 200, 966.8014224022627]
[3477, 200, 967.7878203354776]
[3478, 200, 968.5556355006993]
[3479, 200, 969.514145731926]
[3480, 200, 970.4705542996526]
[3481, 200, 971.4026285558939]
[3482, 200, 972.1767445020378]
[3483, 200, 973.1655677221715]
[3484, 200, 974.0711191855371]
[3485, 200, 975.0631158985198]
[3486, 200, 975.8989573866129]
[3487, 200, 976.8539480865002]
[3488, 200, 977.6951514184475]
[3489, 200, 978.6947894021869]
[3490, 200, 979

In [2]:
data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H6(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/29sept22no1.txt','wb') # H6 series (date typo)
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3611, 200, 0.9904431477189064]
[3612, 200, 1.9553093761205673]
[3613, 200, 2.914745144546032]
[3614, 200, 3.7049881629645824]
[3615, 200, 4.750172100961208]
[3616, 200, 5.607533048838377]
[3617, 200, 6.57969842851162]
[3618, 200, 7.387541461735964]
[3619, 200, 8.386794351041317]
[3620, 200, 9.274644747376442]
[3621, 200, 10.286552716046572]
[3622, 200, 11.048429045826197]
[3623, 200, 11.98720221221447]
[3624, 200, 12.885945782065392]
[3625, 200, 13.890141315758228]
[3626, 200, 14.707734812051058]
[3627, 200, 15.719063300639391]
[3628, 200, 16.573659978806973]
[3629, 200, 17.52584734559059]
[3630, 200, 18.375067181885242]
[3631, 200, 19.32831222936511]
[3632, 200, 20.179290883243084]
[3633, 200, 21.211575150489807]
[3634, 200, 22.001455683261156]
[3635, 200, 23.021300062537193]
[3636, 200, 23.960932414978743]
[3637, 200, 24.942709431052208]
[3638, 200, 25.741677064448595]
[3639, 200, 26.756511464715004]
[3640, 200, 27.69568259641528]
[3641, 200, 28.672302663326263]
[3642, 200, 29.48888

[3872, 200, 242.08703488484025]
[3873, 200, 243.06717606261373]
[3874, 200, 243.84830858185887]
[3875, 200, 244.83107123151422]
[3876, 200, 245.77688103169203]
[3877, 200, 246.70579269900918]
[3878, 200, 247.50702161341906]
[3879, 200, 248.49386068060994]
[3880, 200, 249.41935858130455]
[3881, 200, 250.3691848963499]
[3882, 200, 251.18081511557102]
[3883, 200, 252.14789159968495]
[3884, 200, 253.01557245105505]
[3885, 200, 254.08954121172428]
[3886, 200, 254.88132001832128]
[3887, 200, 255.89456691592932]
[3888, 200, 256.81416423246264]
[3889, 200, 257.7905898466706]
[3890, 200, 258.5753913000226]
[3891, 200, 259.58211889863014]
[3892, 200, 260.4946357309818]
[3893, 200, 261.4614295363426]
[3894, 200, 262.29380375146866]
[3895, 200, 263.3199874833226]
[3896, 200, 264.2071982473135]
[3897, 200, 265.2034072987735]
[3898, 200, 265.98951986804605]
[3899, 200, 266.9751320667565]
[3900, 200, 267.96131819859147]
[3901, 200, 268.9223240837455]
[3902, 200, 269.7036256492138]
[3903, 200, 270.704

[4134, 200, 486.18389907851815]
[4135, 200, 487.17808966338634]
[4136, 200, 488.0888347029686]
[4137, 200, 489.1235497817397]
[4138, 200, 489.9077878817916]
[4139, 200, 490.89722803235054]
[4140, 200, 491.8843919970095]
[4141, 200, 492.89060297980905]
[4142, 200, 493.6902514472604]
[4143, 200, 494.71054366603494]
[4144, 200, 495.60726906359196]
[4145, 200, 496.60835241153836]
[4146, 200, 497.41854574903846]
[4147, 200, 498.4235223941505]
[4148, 200, 499.3424726165831]
[4149, 200, 500.36088029667735]
[4150, 200, 501.20158364996314]
[4151, 200, 502.20601093024015]
[4152, 200, 503.1354568824172]
[4153, 200, 504.0991630330682]
[4154, 200, 504.8891088627279]
[4155, 200, 505.9434230327606]
[4156, 200, 506.8290027268231]
[4157, 200, 507.81532641500235]
[4158, 200, 508.69666814804077]
[4159, 200, 509.6801164858043]
[4160, 200, 510.58898616582155]
[4161, 200, 511.6176836118102]
[4162, 200, 512.4025921821594]
[4163, 200, 513.3797394633293]
[4164, 200, 514.3201956152916]
[4165, 200, 515.382304660

[4400, 200, 735.5925536602736]
[4401, 200, 736.6046443022788]
[4402, 200, 737.3879531808197]
[4403, 200, 738.4026404470205]
[4404, 200, 739.3279295824468]
[4405, 200, 740.3463655598462]
[4406, 200, 741.101551797241]
[4407, 200, 742.1347906291485]
[4408, 200, 743.0353579968214]
[4409, 200, 744.0317612476647]
[4410, 200, 744.9184906482697]
[4411, 200, 745.9304214641452]
[4412, 200, 746.8147088997066]
[4413, 200, 747.8308557979763]
[4414, 200, 748.6130220144987]
[4415, 200, 749.6233489662409]
[4416, 200, 750.5334308296442]
[4417, 200, 751.5393556132913]
[4418, 200, 752.3401022329926]
[4419, 200, 753.3436956144869]
[4420, 200, 754.3126759789884]
[4421, 200, 755.2807847335935]
[4422, 200, 756.1279349662364]
[4423, 200, 757.0889038965106]
[4424, 200, 758.0081619173288]
[4425, 200, 759.0652369633317]
[4426, 200, 759.8483844809234]
[4427, 200, 760.8380919806659]
[4428, 200, 761.7814788632095]
[4429, 200, 762.7622699178755]
[4430, 200, 763.5855076983571]
[4431, 200, 764.6168679334223]
[4432, 20

[4666, 200, 985.4930815473199]
[4667, 200, 986.4918489791453]
[4668, 200, 987.4466446340084]
[4669, 200, 988.4699966348708]
[4670, 200, 989.3181396462023]
[4671, 200, 990.332002196461]
[4672, 200, 991.2083451449871]
[4673, 200, 992.162208661437]
[4674, 200, 993.0186535790563]
[4675, 200, 994.0645941831172]
[4676, 200, 994.9980359151959]
[4677, 200, 996.0193285495043]
[4678, 200, 996.8054430782795]
[4679, 200, 997.7931526489556]
[4680, 200, 998.7880639284849]
[4681, 200, 999.7863577343524]
[4682, 200, 1000.56935948506]
[4683, 200, 1001.6238973140717]
[4684, 200, 1002.53095446527]
[4685, 200, 1003.5367977432907]
[4686, 200, 1004.4085349664092]
[4687, 200, 1005.3851232640445]
[4688, 200, 1006.2824165821075]
[4689, 200, 1007.3079375475645]
[4690, 200, 1008.179874882102]
[4691, 200, 1009.1632654331625]
[4692, 200, 1010.1431990787387]
[4693, 200, 1011.1619459651411]
[4694, 200, 1011.9365207478404]
[4695, 200, 1012.9995401315391]
[4696, 200, 1013.8654350675642]
[4697, 200, 1014.9067701175809]